## BAM (Bottleneck Attention Module)

- SE Block에서 사용한 channel attention에
- 이미지의 spatial한 특징을 학습하는 attention을 추가하여
- 다양한 CNN 모델에 적용이 가능한 모듈

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChannelAttentionModule(nn.Module):
    def __init__(self, in_channels, r=16):
        super(ChannelAttentionModule, self).__init__()

        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // r, bias=True),
            nn.BatchNorm1d(in_channels // r),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // r, in_channels, bias=True),
            nn.BatchNorm1d(in_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        batch_size, channel, _, _ = x.size()

        y = self.squeeze(x).view(batch_size, channel) # input을 globalavg pooling 하여 1d로 변환
        y = self.fc(y).view(batch_size, channel, 1, 1) # 최종 output을 input shape로 변환

        return y.expand_as(x) # (batch_size, channel, H, W)로 맞추기 위해 확장

# class Flatten(nn.Module):
#     def forward(self, x):
#         return x.view(x.size(0), -1)
# class ChannelGate(nn.Module):
#     def __init__(self, gate_channel, reduction_ratio=16, num_layers=1):
#         super(ChannelGate, self).__init__()
#         self.gate_c = nn.Sequential()
#         self.gate_c.add_module( 'flatten', Flatten() )
#         gate_channels = [gate_channel]
#         gate_channels += [gate_channel // reduction_ratio] * num_layers
#         gate_channels += [gate_channel]
#         for i in range( len(gate_channels) - 2 ):
#             self.gate_c.add_module( 'gate_c_fc_%d'%i, nn.Linear(gate_channels[i], gate_channels[i+1]) )
#             self.gate_c.add_module( 'gate_c_bn_%d'%(i+1), nn.BatchNorm1d(gate_channels[i+1]) )
#             self.gate_c.add_module( 'gate_c_relu_%d'%(i+1), nn.ReLU() )
#         self.gate_c.add_module( 'gate_c_fc_final', nn.Linear(gate_channels[-2], gate_channels[-1]) )
#     def forward(self, in_tensor):
#         avg_pool = F.avg_pool2d( in_tensor, in_tensor.size(2), stride=in_tensor.size(2) )
#         return self.gate_c( avg_pool ).unsqueeze(2).unsqueeze(3).expand_as(in_tensor)
    

class SpatialAttentionModule(nn.Module):
    def __init__(self, in_channels, dilation=4, r=16):
        super(SpatialAttentionModule, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // r, kernel_size=1, padding=0),
            nn.BatchNorm2d(in_channels // r),
            nn.ReLU(inplace=True),
        )
        self.conv3_1 = nn.Sequential(
            nn.Conv2d(in_channels // r, in_channels // r, kernel_size=3, padding=dilation, dilation=dilation),
            nn.BatchNorm2d(in_channels // r),
            nn.ReLU(inplace=True),
        )
        self.conv3_2 = nn.Sequential(
            nn.Conv2d(in_channels // r, in_channels // r, kernel_size=3, padding=dilation, dilation=dilation),
            nn.BatchNorm2d(in_channels // r),
            nn.ReLU(inplace=True),
        )
        self.conv1_2 = nn.Sequential(
            nn.Conv2d(in_channels // r, 1, kernel_size=1, padding=0),
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv1_2(x)


        return x.expand_as(x)
    
class BAM(nn.Module):
    def __init__(self, in_channels):
        super(BAM, self).__init__()

        self.channel_attention = ChannelAttentionModule(in_channels)
        self.spatial_attention = SpatialAttentionModule(in_channels)

    def forward(self, x):
        channel_att_map = self.channel_attention(x)
        spatial_att_map = self.spatial_attention(x)

        att_map = 1 + F.sigmoid(channel_att_map + spatial_att_map)

        return x * att_map
    
class BasicBlock(nn.Module) :
    """Basic Block for resnet 18 and resnet 34
    """

    #BasicBlock and BottleNeck block
    #have different output size
    #we use class attribute expansion
    #to distinct

    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()

        # residual function
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), stride=(stride, stride), padding=(1, 1), bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=(3, 3), padding=(1, 1), bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion)
        )

        # shortcut
        self.shortcut = nn.Sequential()

        #the shortcut output dimension is not the same with residual function
        #use 1*1 convolution to match the dimension
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels :
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=(1, 1), stride=(stride, stride), bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))
    

class BottleNeck(nn.Module):
    """Residual block for resnet over 50 layers
    """
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * BottleNeck.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))

    

class ResNet(nn.Module) :
    def __init__(self, block, num_block, num_classes=100, num_channels=3):
        super(ResNet, self).__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(num_channels, 64, kernel_size=(3, 3), padding=(1, 1), bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            BAM(64)
        )
        
        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides :
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        
        if out_channels != 512:
            layers += [BAM(self.in_channels)]
        
        return nn.Sequential(*layers)

    def forward(self, x, feature=False):
        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        if feature: return output
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output)

        return output
    

def resnet18_bam(num_classes, num_channels) :
    """ return a ResNet 18 object
    [?, 3, 224, 224] -> [?, 64, 224, 224]
    [?, 64, 224, 224] -> [?, 64, 224, 224]
    [?, 64, 224, 224] -> [?, 128, 112, 112]
    [?, 128, 112, 112] -> [?, 256, 56, 56]
    [?, 256, 56, 56] -> [?, 512, 28, 28]
    [?, 512, 28, 28] -> [?, 512, 1, 1]
    """
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes, num_channels)

def resnet152_bam(num_classes, num_channels):
    """ return a ResNet 152 object
    [?, 3, 224, 224] -> [?, 64, 224, 224]
    [?, 64, 224, 224] -> [?, 256, 224, 224]
    [?, 256, 224, 224] -> [?, 512, 112, 112]
    [?, 512, 112, 112] -> [?, 1024, 56, 56]
    [?, 1024, 56, 56] -> [?, 2048, 28, 28]
    [?, 2048, 28, 28] -> [?, 2048, 1, 1]
    """
    return ResNet(BottleNeck, [3, 8, 36, 3], num_classes, num_channels)

In [6]:
if __name__ == '__main__':
    model = resnet152_bam(num_classes=1000, num_channels=3)
    inp = torch.rand(2, 3, 224, 224)
    oup = model(inp)
    print(oup.shape)
    predicted_labels = oup.argmax(dim=1)
    print(predicted_labels) 

torch.Size([2, 1000])
tensor([774, 774])
